$L = K_0 \times \dotsb K_m$, where each $K_i/k$ is a cyclic extension of number field with degree $p^{\epsilon_i}$. These fields satisfies that
* $K_i \not\subset K_j$ for all $i, j$.
* $\cap_{i \in \mathcal{I}'} K_i = k$.
* $e_i \geq e_{i+1}$.

# Special case
Let $k = \mathbb(Q)(\zeta_{p^n})$. Let $l_1, l_2$ be distinct prime numbers which are not equal to $p$. Each $K_i$ is given by a pair of numbers $(a_i, b_i)$ such that $0 \leq a_i < p^n$, $0 \leq b_i < p^n$, and $(a_i, b_i) \neq (0, 0)$, which gives
\begin{align*}
K_i = k\left((l_1^{a_i} l_2^{b_i})^{1/p^n}\right)
\end{align*}

Input: the cyclic extensions $K_1, \dotsc K_m$ and $k$.

Ouput: the groups $Ш^1(k, T_{L/k})$ and $Ш^2_{\omega}(k, \hat{T}_{L/k})$.

# Inputs

In [1]:
m = 4
p = 3
n = 3
l1 = 5
l2 = 19
a = [1, 1, 2, 3, 5]
b = [0, 1, 3, 5, 11]
# k = CyclotomicField(p^n)
# K = [k.extension(x^(p^n) - l1^a[i] * l2^b[i], 'a' + str(i)) for i in range(0, m+1)]

In [2]:
v_p = QQ.valuation(p)

In [3]:
# compute m12 and m21
s1 = v_p(l1-1)
s2 = v_p(Mod(l2, l1).multiplicative_order())
s1prime = v_p(l2-1)
s2prime = v_p(Mod(l1, l2).multiplicative_order())
m12 = max(min(n, s1)-(s1-s2), 0)
m21 = max(min(n, s1prime)-(s1prime-s2prime), 0)

In [4]:
I = [i for i in range(1, m+1)]       # a list, {1, 2, ..., m}
I_prime = [i for i in range(0, m+1)] # a list, {0, 1, 2, ..., m}

Find $\epsilon_i$, where $p^{\epsilon_i} = [K_i:k]$.

In [5]:
epsilon = [0]*(m+1)
for i in I_prime:
    epsilon[i] = n-min(v_p(a[i]), v_p(b[i]))
epsilon

[3, 3, 3, 3, 3]

In [6]:
def is_subfield(F1, F2):
    """
    Check that if F1 is a subfield of F2, assuming that F1 is Galois.
    
    Parameters
    ----------
    F1 : sage.rings.number_field.number_field
        A number field whose base field can be Q or another number field.
    F2 : sage.rings.number_field.number_field
        A number field whose base field can be Q or another number field.
        
    Returns
    ---------
    result : boolean
        True iff. F1 is contained in F2.
    """
    F1.<f1> = F1.absolute_field()
    F2.<f2> = F2.absolute_field()
    R.<x> = PolynomialRing(F2) # R = F2[x]
    f = R(F1.polynomial())     # Let f be the minimal polynomial of F1/Q, and regard f as in R = F2[x].
    l = list(f.factor())       # factorize f in R
    
    result = True
    for factor in l:
        # check if each factor of f is of degree 1,
        # if not, then F1 is not a subfield of F2
        if factor[0].degree() != 1:
            result = False
            break
    return result

Find $e_i$ and $e_{i, j}$, where $p^{e_i} = [K_0 K_i : K_i]$ and $p^{e_{i, j}} = [K_i \cap K_j: k]$.

In [7]:
e_ij = [
    [v_p(gcd([a[i] - a[j], b[i] - b[j], p^n])) for j in range(0, m+1)] 
    for i in range(0, m+1)
]

In [8]:
e_ij

[[3, 0, 0, 0, 0],
 [0, 3, 0, 0, 0],
 [0, 0, 3, 0, 0],
 [0, 0, 0, 3, 0],
 [0, 0, 0, 0, 3]]

In [9]:
e_i = [epsilon[0] - e_ij[0][i] for i in range(0, m+1)]

In [10]:
e_i

[0, 3, 3, 3, 3]

$R = \{i \in \mathcal{I}'\mid U_i \neq \varnothing\}$, where $U_r = \{i \in \mathcal{I} \mid e_{0, i} = r\}$.

In [11]:
def U(r):
    """
    Return the set U_r.
    
    Parameters
    ----------
    r : int
        The number in U_r.
    
    Returns
    ----------
    result : list
        A list consisting of indicies i such that e_{0, i} = r.
    """
    
    result = []
    for i in I:
        if e_ij[0][i] == r:
            result.append(i)
    return result

R = []
for i in range(epsilon[0]+1):
    if len(U(i)) > 0:
        R.append(i)

In [12]:
R

[0]

In [13]:
U(0)

[1, 2, 3, 4]

$L(U_r) = \min\{\log_p[K_i \cap K_j: k] \mid \forall i, j \in U_r\}$

In [14]:
def L(U):
    """
    For a subset U of I', return L(U).
    
    Parameters
    ----------
    U : list or tuple
        A subset of I' = {0, 1, ..., m}
    
    Returns
    ---------
    int
        The minimal integer of e_ij for i, j in U, 
        where p^{e_ij} is the degree of the intersection of K_i and K_j over k.
    """

    r = []
    for i in U:
        for j in U:
            r.append(e_ij[i][j])
    if len(r) > 0:
        return min(r)
    else:
        return 'The set is empty!'

In [15]:
L(U(0))

0

On a subset $C \subset \mathcal{I}'$, we can define an equivalence relation $\sim_{\ell}$ by
$$i \sim_{\ell} j \Leftrightarrow e_{ij} \geq l.$$
The number $n_{\ell}(C)$ is the number of equivalence classes with respect to $C/\sim_{\ell}$.

In [15]:
def equiv(i, j, l):
    """
    Return whether i and j are l-equivalent.
    
    Parameters
    ----------
    i, j : int
        Two indicies in I'.
    l : int
        The equivalence relation we consider.
        
    Returns
    ---------
    bool
        True iff i and j are l-equivalent.
    """
    return e_ij[i][j] >= l or i == j

In [16]:
def equiv_classes(C, l):
    """
    Return a list of equivalence classes with respect to C/l.
    
    Parameters
    ----------
    C : list or tuple
        A subset of I_prime whose l-equivalence class we consider.
    l : int
        The equivalence relation we consider.
        
    Returns
    ---------
    components : a list of lists of vertices.
        Each components[i] consists of vertices which are l-equivalent.
    """
    
    # Construct a graph whose the vertices are elements in C, 
    # and two vertices i and j are connected with an edge iff i~j.
    d = {}
    for i in C:
        # For a vertice i,
        # collect those vertices j which are l-equivalent to i.
        connected_vertices = []
        for j in C:
            if i != j and equiv(i, j, l):
                connected_vertices.append(j)
        d[i] = connected_vertices
    G = Graph(d)
    # Use the built-in function G.connected_components() in Sage.
    return G.connected_components()

In [17]:
def number_of_equiv_class(l, C):
    """
    Return the number of equivalence classes of C/~l.
    
    Parameters
    ----------
    l : int
        The equivalence relation we consider.
    C : list or tuple
        A subset of I_prime whose l-equivalence class we consider.
        
    Returns
    ---------
    n : int
        Number of equivalence classes of C/~l.
    """
    
    # Construct a graph whose the vertices are elements in C, 
    # and two vertices i and j are connected with an edge iff i~j.
    d = {}
    for i in C:
        connected_vertices = []
        for j in C:
            if i != j and equiv(i, j, l):
                connected_vertices.append(j)
        d[i] = connected_vertices
    G = Graph(d)
    # Use the built-in function G.connected_components_number() in Sage.
    return G.connected_components_number()

For a cyclic extension $K_i$ of degree $p^{\epsilon_i}$, we denote by $K_i(f)$ the unique subfield of $K_i$ of degree $p^f$ (p4).

In [18]:
def K_cyclic_subfield(i, d):
    """
    Return the cyclic subfield K_i(d).
    
    Parameters
    ----------
    i : int
        The index of the field K_i which we are interested in.
    d : int
        The power of degree p^d.
    
    Returns
    ---------
    K_i_d : a relative number field
        The unique subfield of K_i/k of degree p^d. It is a relative number field in Sage whose base field is k.
    """
    return k.extension(x^(p^d) - l1^a[i] * l2^b[i], 'a' + str(i) + '_' + str(d))

For a nonempty subset $C \subseteq \mathcal{I}$ and an integer $d \geq 0$, we define the field $M_C(d)$ to be the composite field $\langle K_i(d) \rangle_{i \in C}$.

In [19]:
def M_composite(C, d):
    """
    Return the composite field M_C(d).
    
    Parameters
    ----------
    C : list or tuple
        A subset of I.
    d : int
        A nonnegative integer. We concern about p^d.
    
    Returns
    ---------
    result : sage.rings.number_field.number_field.NumberField_absolute_with_category
        The composite of the fields K[i](d).
    """
    
    result = k
    for i in C:
        # composite one field at a time
        result = result.composite_fields(K_cyclic_subfield(i, d))[0]
    result = k.composite_fields(result)[0]
    return result

## Definition 4.1.
The *algebraic patching degree* $\Delta^{\omega}_r$ of $U_r$ is the maximum nonnegative integer $d$ satisfying the following:
1. If $U_{>r}$ is nonempty, then $M_{U_{>r}}(d) \subset \cap_{i \in U_r} K_0(d) K_i(d)$.
2. If $U_{<r}$ is nonempty, then $M_{U_r}(d) \subset \cap_{i \in U_{<r}} K_0(d) K_i(d)$.

If $U_r = \mathcal{I}$, then we set $\Delta^{\omega}_r = \epsilon_0$.

Here $U_{> r} = \{i \in \mathcal{I} \mid e_{0, i} > r\}$ and $U_{< r} = \{i \in \mathcal{I} \mid e_{0, i} < r\}$. (p.13)

In [20]:
def U_greater(r):
    """
    Return the set U_{>r}.
    
    AParameters
    ----------
    r : int
    
    Returns
    ---------
    U_{>r} : list
        A subset of I consisting of indicies i such that e_0i > r.
    """
    result = []
    for i in I:
        if e_ij[0][i] > r:
            result.append(i)
    return result

def U_smaller(r):
    """
    Return the set U_{<r}.
    
    Parameters
    ----------
    r : int
    
    Returns
    ---------
    U_{<r} : list
        A subset of I consisting of indicies i such that e_0i > r.
    """
    result = []
    for i in I:
        if e_ij[0][i] < r:
            result.append(i)
    return result

In [21]:
def Delta_omega(r):
    """
    Return the algebraic patching degree of U_r. (Definition 4.1)
    
    Parameters
    ----------
    r : int
    
    Returns
    ---------
    d : int
        The algebriac patching degree of U_r.
    """
    
    U_r = U(r)
    
    # trivial case
    if U_r == I:
        return epsilon[0]
    
    # non-trivial case
    U_greater_r = U_greater(r)
    U_smaller_r = U_smaller(r)
    
    # whether we should check two conditions
    flag_1 = bool(len(U_greater(r)) > 0) # U_{>r} is nonempty
    flag_2 = bool(len(U_smaller(r)) > 0) # U_{<r} is nonempty
    
    for d in range(r, epsilon[0]+1):
        d_satisfy = True # check if the current d satisfies two conditions
        
        if flag_1: # 1. If U_{>r} is nonempty,
            G = AbelianGroup([p^n, p^n])
            H1 = []
            for j in U_greater_r:
                H1.append(G((a[j]*p^(epsilon[j]-d), b[j]*p^(epsilon[j]-d))))
            H1 = G.subgroup(H1)
            
            for i in U_r:
                H2 = G.subgroup([
                    G((a[0]*p^(epsilon[0]-d), b[0]*p^(epsilon[0]-d))), 
                    G((a[i]*p^(epsilon[i]-d), b[i]*p^(epsilon[i]-d)))
                ])
                if not H1.is_subgroup(H2):
                    d_satisfy = False
                    break
                
#                 temp_field = K_cyclic_subfield(0, d).composite_fields(K_cyclic_subfield(i, d))[0]
#                 # check if M_{U_{>r}}(d) is contained in K_0(d)K_i(d).
#                 if not is_subfield(M_composite(U_greater_r, d), temp_field):
#                     d_satisfy = False
#                     break
        if flag_2: # 2. If U_{<r} is nonempty,
            G = AbelianGroup([p^n, p^n])
            H1 = []
            for j in U_r:
                H1.append(G((a[j]*p^(epsilon[j]-d), b[j]*p^(epsilon[j]-d))))
            H1 = G.subgroup(H1)
            
            for i in U_smaller_r:
                H2 = G.subgroup([
                    G((a[0]*p^(epsilon[0]-d), b[0]*p^(epsilon[0]-d))), 
                    G((a[i]*p^(epsilon[i]-d), b[i]*p^(epsilon[i]-d)))
                ])
                if not H1.is_subgroup(H2):
                    d_satisfy = False
                    break
                
#                 temp_field = K_cyclic_subfield(0, d).composite_fields(K_cyclic_subfield(i, d))[0]
#                 # check if M_{U_r}(d) is contained in K_0(d)K_i(d).
#                 if not is_subfield(M_composite(U_r, d), temp_field):
#                     d_satisfy = False
#                     break
        if not d_satisfy:
            return d-1
    return epsilon[0]

In [22]:
Delta_omega(0)

3

convert absolute field to relative number field

In [23]:
def abs_to_rel(F):
    """
    Given an absolute number field F, return the same relative number field whose base field is k. 
    
    Parameters
    ----------
    F : sage.rings.number_field.number_field.NumberField_absolute_with_category
        An absolute number field which we assumed to be Galois and contains k.
        
    Returns
    ---------
    F_rel : sage.rings.number_field.number_field_rel.NumberField_relative_with_category
        A relative number field in Sage whose base field is k.
    """
    
    if not is_subfield(k, F):
        raise ValueError(f"F is not an extension of k. In this case, F is {F}.")
    
    R.<x> = PolynomialRing(k)        # R = k[x]
    p = R(F.absolute_polynomial())            # the minimal polynomial of F, viewed as an element in R = k[x]
    l = list(p.factor())             # factorize p in k[x]
    F_rel.<v> = k.extension(l[0][0]) 
    return F_rel

A finite Galois extension $F$ of $k$ is said to be *locally cyclic* at $v$ if $F \otimes_k k_v$ is a product of cyclic extensions of $k_v$. $F$ is said to be *locally cyclic* if it is locally cyclic at all $v \in \Omega_k$. (p2)

We divide this problems into two parts. Note that if a place $v$ of $k$ is unramified, then $F$ is locally cyclic at $v$. Thus it remains to check on ramified places. 
1. How to find ramified places?
    
    Let $\mathfrak{p}$ prime ideal of $k$. $\mathfrak{p}$ is ramified in $F$ iff $\mathfrak{p}$ divides the relative discriminant $\Delta_{F/k}$.
    
    
2. How to check if $F$ is locally cyclic at a ramified place $v$?

    Since $F/k$ is Galois, for any $w \mid v$ and $w' \mid v$ the completions $F_{w} \simeq F_{w'}$ are isomorphic. Thus to check that $F/k$ is locally cyclic at $v$, it suffices to check whether $F_w/k_v$ is cyclic where $w$ is a place of $F$ lying over $v$.

In [24]:
def is_locally_cyclic(F):
    """
    Check if a finite Galois extension F/k is locally cyclic.
    
    Parameters
    ----------
    F : sage.rings.number_field.number_field.NumberField_absolute_with_category
        An absolute number field which we assumed to be Galois and contains k.
        
    Returns
    ---------
    result : bool
        True iff. F/k is locally cyclic.
    """
    
    if not is_subfield(k, F):
        raise ValueError(f"F is not an extension of k. In this case, F is {F}.")
    
    # check if F == k
    if is_subfield(F, k):
        return True
    
    # Part 1: find ramified places.
    
    F = abs_to_rel(F)
    
    # the absolute number field of F
    F_abs.<w> = F.absolute_field()
    # replace F with a relative number field with base field k
    F = abs_to_rel(F_abs)
    # Take a generator of k/Q, viewed in F_abs
    k_gen = (k.polynomial()).roots(ring = F_abs, multiplicities = False)[0]
    # Galois group F/Q
    G_abs = F_abs.galois_group()
    # relative discriminant of F/k, an ideal of k
    rel_disc = k.ideal(F.relative_discriminant())
    
    # factorize the relative discriminant in F
    # temp_list[0] = (P, n), where P: prime ideal, n: power
    temp_list = list(F.ideal(rel_disc).factor()) 
    # we only need the P part
    ramified_primes_above = [item[0] for item in temp_list]
    
    #------------------------------------------------------
    # Part 2: check for each ramified place.
    result = True
    for P in ramified_primes_above:
        try:
            # P is an ideal in relative number field F.
            # Convert P to an ideal in absolute number field F_abs.
            P_abs = P.absolute_ideal(w)
        except:
            print(F)
            print(F.base_field())
        
        # the decomposition group of P in F/Q
        abs_decomp = G_abs.decomposition_group(P_abs)
        # compute the decomposition group of P in F/k
        rel_decomp = []
        for g in abs_decomp:
            # check if an element g in rel_decomp fixes k
            if g(k_gen) == k_gen:
                rel_decomp.append(g)
        # convert rel_decomp into a subgroup
        rel_decomp = abs_decomp.subgroup(rel_decomp)
        # check if the (relative) decomposition group is cyclic
        if not rel_decomp.is_cyclic():
            result = False
            break
    return result

$M_c(d)$ is locally cyclic \
$\Leftrightarrow p^{n-d} c_1(c) \in p^{m_{21}} \mathbb{Z}$ and $p^{n-d} d_2(c) \in p^{m_{12}} \mathbb{Z}$.

Here
\begin{align*}
m_{12} = \max\{\min\{n, s_1\} - (s_1 - s_2), 0\} \\
s_1 = v_p(l_1 - 1), s_2 = v_p(\mathrm{ord}[l_2]_{l_1}) \\
m_{21} = \max\{\min\{n, s'_1\} - (s'_1 - s'_2), 0\} \\
s'_1 = v_p(l_2 - 1), s'_2 = v_p(\mathrm{ord}[l_1]_{l_2}).
\end{align*}
Also,
\begin{align*}
\langle (c_1(c), d_1(c)), (0, d_2(c)) \rangle = \langle (a'_i, b'_i) \mid i \in c \rangle
\end{align*}
where
\begin{align*}
(a_i, b_i) = p^{n - \epsilon_i}(a'_i, b'_i)
\end{align*}

In [25]:
def M_locally_cyclic(c, d):
    try:

        # compute c_1(c) and d_2(c)
        G = AbelianGroup([p^n, p^n])
        H = []
        for i in c:
            H.append(G((a[i]//p^(n-epsilon[i]), b[i]//p^(n-epsilon[i]))))
        H = G.subgroup(H)
        c_list = []
        d_list = []
        for g in G:
            cg = G.subgroup([g])
            if cg.is_subgroup(H): # g is in H
                c_list.append(g.list()[0])
                if g.list()[0] == 0:
                    d_list.append(g.list()[1])
#         for h in H:
#             # convert h to an element in G
#             h_as_G = G(1)
#             for k in range(len(H.gens())):
#                 h_as_G = h_as_G * G(tuple(H.gens()[k].list()))^h.list()[k]

#             c_list.append(h_as_G.list()[0])
#             if h_as_G.list()[0] == 0:
#                 d_list.append(h_as_G.list()[1])
        c_1_c = gcd(c_list)
        d_2_c = gcd(d_list)
        return (p^(n-d)*c_1_c) % (p^m21) == 0 and (p^(n-d)*d_2_c) % (p^m12) == 0
    except:
        print(f"Error happened. The inputs were c = {c}, d = {d}")

In [26]:
M_locally_cyclic([2, 3, 0], 1)

True

## Definition 4.9.
Define the *patching degree* $\Delta_r$ of $U_r$ to be the maximum nonnegative integer $d \leq \Delta^{\omega}_r$ satisfying the following:
1. If $U_{>r}$ is nonempty, then the field $K_0(d) M_{U_{> r}}(d)$ is locally cyclic.
2. If $U_{<r}$ is nonempty, then the field $K_0(d) M_{U_r}(d)$ is locally cyclic.

If $U_r = \mathcal{I}$, we set $\Delta_r = \epsilon_0$.
We have $\Delta^{\omega}_r \geq \Delta_r \geq r$.

In [27]:
def Delta(r):
    """
    Return the patching degree of U_r.
    
    Parameters
    ----------
    r : int
        An integer in the set R.
        
    Returns
    ----------
    d : int
        The patching degree $\Delta_r$ of the set $U_r$.
    """
    
    U_r = U(r)
    
    # trivial case
    if U_r == I:
        return epsilon[0]
    
    # non-trivial case
    
    # some constants used later
    Delta_omega_r = Delta_omega(r)      # the algebraic patching degree $\Delta^{\omega}_r$
    U_greater_r = U_greater(r)          # the set $U_{>r}$
    U_smaller_r = U_smaller(r)          # the set $U_{<r}$
    
    # whether we should check two conditions
    flag_1 = bool(len(U_greater_r) > 0) # $U_{>r}$ is nonempty
    flag_2 = bool(len(U_smaller_r) > 0) # $U_{<r}$ is nonempty
    
    # Check for each d <= Delta_omega_r
    # If d does not satisfy, then d+1 does not, either.
    # Thus we let d run from r to Delta_omega_r;
    # return d-1 if d does not satisfy the conditions;
    # return Delta_omega_r if each d satisfy.
    for d in range(r, Delta_omega_r+1):
        d_satisfy = True
        
        if flag_1: # 1. If $U_{>r}$ is nonempty,
            if not M_locally_cyclic(U_greater_r + [0], d):
                d_satisfy = False
            
        if flag_2: # 2. If $U_{<r}$ is nonempty,
            if not M_locally_cyclic(U_r + [0], d):
                d_satisfy = False
                
        if not d_satisfy:
            return d-1
    return Delta_omega_r

A *bicyclic extension* $F/k$ is a Galois extension with $\mathrm{Gal}(F/k)$ isomorphic to $\mathbb{Z}/n_1 \mathbb{Z} \times \mathbb{Z}/n_2 \mathbb{Z}$ where $n_1, n_2 > 1$ and $n_2 \mid n_1$. (p2)

I think that this can be checked by looking at the numbers of elementary divisors of $\mathrm{Gal}(F/k)$: if there are more than $2$ elementary divisor then $F/k$ is not a bicyclic extension.

In [29]:
def is_bicyclic(F):
    """Check if F is a subfield of a bicyclic extension of k.
    
    Parameters
    ----------
    F : sage.rings.number_field.number_field_rel.NumberField_relative_with_category
        A number field whose base field is k.
    
    Returns
    ----------
    result : bool
        True if F is a subfield of a bicyclic extension of k. False if F is not.
    """
    
    # convert F to an absolute number field
    L.<w> = F.absolute_field()
    # compute Gal(F/Q)
    G = L.galois_group()
    # a generator of k/Q, which is an element in L = F.absolute_field()
    k_gen = k.polynomial().roots(ring = L, multiplicities = False)[0]
    
    # compute Gal(F/k)
    k_gal = []
    for g in G:
        try:
            if g(k_gen) == k_gen: # check if the automorphism g fixes k
                k_gal.append(g)
        except:
            print(f"I can't handle when F is the field {F}.")
    Gal_F_k = G.subgroup(k_gal)
    
    
    if Gal_F_k.is_abelian(): # check if F/k is abelian.
        
        # Gal_F_k is implemented as a subgroup of permutation group.
        # Find a reduced finite presentation of Gal_F_k.
        Gal_fin_present = Gal_F_k.as_finitely_presented_group(reduced = True)
        
        # Find the abelian invariants of Gal_F_k.
        gen_orders = [a for a in Gal_fin_present.abelian_invariants()]
        
        # F is a subfield of a bicyclic extension of k 
        # <=>
        # Gal(F/k) is abelian and has <= 2 invariant factors.
        return len(AbelianGroup(gen_orders).elementary_divisors()) <= 2
    
    else:
        return False

## Definition 5.3
For a nonempty $U_r$, let $l_r = L(U_r)$. Let $f \leq \Delta^{\omega}_r$ be a nonnegative integer satisfying the following:
1. The field $M_{U_r}(f + l_r - r)$ is a subfield of a bicyclic extension.
2. $K_0(f) \subset M_{U_r}(f + l_r - r)$.

Then we set $f^{\omega}_{U_r}$ to be the largest $f \leq \Delta^{\omega}_r$ satisfying above conditions. We call $f^{\omega}_{U_r}$ the *algebraic degree of freedom* of $U_r$.

## Remark 5.4
We have $\Delta^{\omega}_r \geq f^{\omega}_{U_r} \geq r$.

For $h \geq L(U_r)$ and a class $c_0$ of $U_r/\sim_h$, we define by recursion *the algebraic degree of freedom* of $c \in c_0/\sim_{h+1}$ as follows.
## Definition 5.5
Keep the notation defined as above. Let $f \leq f^{\omega}_{c_0}$ be a non-negative integer satisfying the following:
1. The field $M_c(f + L(c) - r)$ is a subfield of a bicyclic extension.
2. $K_0(f) \subset M_c(f + L(c) - r)$.

Then we set $f^{\omega}_c$ to be the largest $f \leq f^{\omega}_{c_0}$ satisfying above conditions. We call $f^{\omega}_c$ the *algebraic degree of freedom* of $c$.

In [30]:
# the dictionary of algebraic degree of freedom
# key
# ---------
# c : tuple
#     A subset that is an equivalent class of U_r/~h for some h.
#
# value
# ---------
# f_omega_c[c] : int
#     The algebraic degree of freedom of c.
f_omega_c = {}

In [31]:
def alg_deg_of_freedom(c, r, c0 = None, compute_next_level = True):
    """Function that constructs the dictionary f_omega_c of algebraic degreee of freedom.
    
    Parameters
    ----------
    c : list
        An equivalent class of U_r/~h for some integer h. We want to compute its algebraic degree of freedom.
    r : int
        The index of the set U_r.
    c_0 : list
        An equivalent class of U_r/~(h-1) for some integer h, and c is an equivalent class of c_0/h.
        If c_0 == None then c is some U_r.
    """
    
    # case 1: c == U_r for some r
    if c0 == None:
        U_r = c
        
        # check if U_r is nonempty
        if len(U_r) > 0:
            l_r = L(U_r)
            Delta_omega_r = Delta_omega(r)
            
            f_omega_c[tuple(set(U_r))] = Delta_omega_r
            for f in range(r, Delta_omega_r + 1):
                
                # check temp_field can be defined
                if min([epsilon[i] for i in U_r]) < f + l_r - r:
                    f_omega_c[tuple(set(U_r))] = f-1
                    break
                    
                # the field $M_{U_r}(f + l_r - r)$.
                # temp_field = M_composite(U_r, f + l_r - r)
                # check if 
                # (1) temp_field is a subfield of a bicylic extension of k (always true in this special case)
                # (2) K_0(f) is a subfield of temp_field
                G = AbelianGroup([p^n, p^n])
                d = f + l_r - r
                W_0_f = G.subgroup([G((a[0]*p^(epsilon[0]-f), b[0]*p^(epsilon[0]-f)))])
                W_c_d = []
                for i in c:
                    W_c_d.append(G((a[i]*p^(epsilon[i]-d), b[i]*p^(epsilon[i]-d))))
                W_c_d = G.subgroup(W_c_d)
                if W_0_f.is_subgroup(W_c_d):
                    continue
                else:
                    f_omega_c[tuple(set(U_r))] = f-1
                    break
                
#                 a_gcd = gcd([a[j] for j in U_r])
#                 b_gcd = gcd([b[j] for j in U_r])
#                 if (a_gcd == 0 and a[0] != 0) or (b_gcd == 0 and b[0] != 0):
#                     f_omega_c[tuple(set(U_r))] = f-1
#                     break
#                 if a[0] % a_gcd == 0 and b[0] % b_gcd == 0:
#                     continue
#                 else:
#                     f_omega_c[tuple(set(U_r))] = f-1
#                     break
            
            if compute_next_level:
                # compute the next nevel
                # if U_r contains only one element then there is no need to compute.
                if len(U_r) > 1:
                    # l <= h iff. c/~l contains only one equivalence class
                    h = l_r
                    for cl in equiv_classes(c, h+1):
                        alg_deg_of_freedom(cl, r, c)
            
    # case 2: c0 is a class of U_r
    if c0 != None:
        
        # check if c is empty
        if len(c) > 0:
            L_c = L(c)
            f_omega_c[tuple(set(c))] = f_omega_c[tuple(set(c0))]
            
            for f in range(1, f_omega_c[tuple(set(c0))] + 1):
                
                # check temp_field can be defined:
                if min([epsilon[i] for i in c]) < f + L_c - r:
                    f_omega_c[tuple(set(c))] = f-1
                    break
                
                # the field $M_c(f + L(c) - r)$
                # temp_field = M_composite(c, f + L_c - r)
                # check if
                # (1) temp_field is a subfield of a bicylic extension (always true in this special case)
                # (2) K_0(f) is a subfield of temp_field
                G = AbelianGroup([p^n, p^n])
                d = f + L_c - r
                W_0_f = G.subgroup([G((a[0]*p^(epsilon[0]-f), b[0]*p^(epsilon[0]-f)))])
                W_c_d = []
                for i in c:
                    W_c_d.append(G((a[i]*p^(epsilon[i]-d), b[i]*p^(epsilon[i]-d))))
                W_c_d = G.subgroup(W_c_d)
                if W_c_d.is_subgroup(W_0_f):
                    continue
                else:
                    f_omega_c[tuple(set(c))] = f-1
                    break
                
#                 a_gcd = gcd([a[j] for j in c])
#                 b_gcd = gcd([b[j] for j in c])
#                 if (a_gcd == 0 and a[0] != 0) or (b_gcd == 0 and b[0] != 0):
#                     f_omega_c[tuple(set(c))] = f-1
#                     break
#                 if a[0] % a_gcd == 0 and b[0] % b_gcd == 0:
#                     continue
#                 else:
#                     f_omega_c[tuple(set(c))] = f-1
#                     break
                    
            if compute_next_level:
                # compute the next level
                # if c contains only one element then there is no need to compute
                if len(c) > 1:
                    # l <= L(c) iff. c/~l contains only one equivalence class
                    h = L_c
                    for cl in equiv_classes(c, h+1):
                        alg_deg_of_freedom(cl, r, c)

In [32]:
# construct the complelte dictionary
for r in range(0, epsilon[0]+1):
    alg_deg_of_freedom(c = U(r), r = r)

In [33]:
f_omega_c

{(1, 2, 3, 4): 3, (1,): 0, (2,): 0, (3,): 0, (4,): 0}

## Definition 5.6
For a nonempty $U_r$, let $h \geq L(U_r)$ and $c \in U_r/\sim_h$. We define the *degree of freedom* $f_c$ to be the maximum integer $f \leq f^{\omega}_c$ such that $M_c(f + L(c) - r)$ is locally cyclic.

In [34]:
# the dictionary of degree of freedom
# key
# ---------
# c : tuple
#     A subset that is an equivalent class of U_r/~h for some h.
#
# value
# ---------
# f_c[c] : int
#     The degree of freedom of c.
f_c = {}

In [35]:
def deg_of_freedom(c, r, compute_next_level = True):
    """Function that constructs the dictionary f_c of degreee of freedom.
    
    Parameters
    ----------
    c : list
        An equivalent class of U_r/~h for some integer h. We want to compute its algebraic degree of freedom.
    r : int
        The index of the set U_r.
    """
    
    # check if c is empty
    if len(c) > 0:
        # convert c to a tuple
        c = tuple(set(c))
        L_c = L(c)
        
        # set default value
        f = f_omega_c[c]
        # check for each f
        while(True):
            if M_locally_cyclic(c, f + L_c - r):
                f_c[c] = f
                break
            else:
                f -= 1
        
        if compute_next_level:
            # compute the next level
            if len(c) > 1:
                # l <= L(c)  iff. c/~l contains only one equivalence class
                h = L_c
                for cl in equiv_classes(c, h+1):
                    deg_of_freedom(cl, r)

In [36]:
# construct the complete dictionary
for r in range(0, epsilon[0]+1):
    deg_of_freedom(U(r), r)

In [37]:
f_c

{(1, 2, 3, 4): 1, (1,): -2, (2,): -2, (3,): -1, (4,): -2}

In [47]:
for i in range(1, 5):
    print(L([i]))

3
3
3
3


## Theorem 6.5
We have
\begin{align*}
Ш^2_{\omega}(k, \hat{T}_{L/K}) \simeq \bigoplus\limits_{r \in \mathcal{R} \setminus \{0\}} \mathbb{Z}/p^{\Delta_r^{\omega} - r} \mathbb{Z} \bigoplus\limits_{r \in \mathcal{R}}
\bigoplus\limits_{l \geq L(U_r)} \bigoplus\limits_{c \in U_r/\sim_l} (\mathbb{Z}/p^{f^{\omega}_c - r} \mathbb{Z})^{n_{l+1}(c) -1}
\end{align*}
and
\begin{align*}
Ш^1(k, T_{L/K}) \simeq \bigoplus\limits_{r \in \mathcal{R} \setminus \{0\}} \mathbb{Z}/p^{\Delta_r - r} \mathbb{Z} \bigoplus\limits_{r \in \mathcal{R}} \bigoplus\limits_{l \geq L(U_r)} \bigoplus\limits_{c \in U_r/\sim_l} (\mathbb{Z}/p^{f_c - r} \mathbb{Z})^{n_{l+1}(c) -1}
\end{align*}

In [40]:
def sha2omega():
    """
    Return the group Ш^2_w(k, ~T_{L/K}).
    
    Returns
    ----------
    G : Multiplicative Abelian Group in Sage.
    """
    
    # In Sage, AbelianGroup([0,0,0,2,3]) means
    # Multiplicative Abelian group isomorphic to Z x Z x Z x C2 x C3.
    
    # the list containing p^{Delta^{omega}_r - r}
    first_part = []
    for r in R:
        if r == 0:
            continue
        if Delta_omega(r)-r >= 0:
            first_part.append(p^(Delta_omega(r)-r))
            
    # the list containing p^{f_c - r}
    second_part = []
    # 1st direct sum, r in R\{0}
    for r in R:
        U_r = U(r)
        l = L(U_r)
        keep_running = True
        # 2nd direct sum, l >= L(U_r)
        while(keep_running):
            # 3rd direct sum, c in U_r/~l
            for c in equiv_classes(U_r, l):
                if f_omega_c[tuple(set(c))] - r >= 0:
                    # there are (n(l+1, c) - 1) copies of Z/ p^{f^{omega}_c - r} Z
                    second_part += [p^(f_omega_c[tuple(set(c))] - r)] * (number_of_equiv_class(l+1, c)-1)
                    
            if number_of_equiv_class(l, U_r) == len(U_r): # there is no need to use finer partition
                keep_running = False
            l += 1
    return AbelianGroup(first_part + second_part)

In [41]:
sha2omega()

Multiplicative Abelian group isomorphic to C27 x C27 x C27

In [42]:
def sha1():
    """
    Return the group Ш^1(k, T_{L/K}).
    
    Returns
    ----------
    G : Multiplicative Abelian Group in Sage.
    """
    
    # In Sage, AbelianGroup([0,0,0,2,3]) means
    # Multiplicative Abelian group isomorphic to Z x Z x Z x C2 x C3.
    
    # the list containing p^{Delta_r - r}
    first_part = []
    for r in R:
        if r == 0:
            continue
        if Delta(r)-r >= 0:
            first_part.append(p^(Delta(r)-r))

    # the list containing p^{f_c - r}
    second_part = []
    # 1st direct sum, r in R\{0}
    for r in R:
        U_r = U(r)
        l = L(U_r)
        keep_running = True
        # 2nd direct sum, l >= L(U_r)
        while(keep_running):
            # 3rd direct sum, c in U_r/~l
            for c in equiv_classes(U_r, l):
                c = tuple(set(c))
                if f_c[c] - r >= 0:
                    # there are (n(l+1, c) - 1) copies of Z/ p^{f_c - r} Z
                    second_part += [p^(f_c[c]-r)]*(number_of_equiv_class(l+1, c)-1)
            if number_of_equiv_class(l, U_r) == len(U_r): # there is no need to use finer partition
                keep_running = False
            l += 1
    return AbelianGroup(first_part + second_part)

In [43]:
sha1()

Multiplicative Abelian group isomorphic to C3 x C3 x C3